# Vitessce integration

For more comprehensive integration tests, see: https://github.com/laminlabs/lamin-spatial

In [ ]:
!lamin login testuser1
!lamin init --storage "s3://lamindb-ci/test-vitessce"

In [ ]:
import lamindb as ln
from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
)

In [ ]:
pbmc68k = ln.core.datasets.anndata_pbmc68k_reduced()

In [ ]:
pbmc68k = pbmc68k[:100, :200].copy()

In [ ]:
zarr_filepath = "my_test.zarr"
pbmc68k.write_zarr(zarr_filepath)

In [ ]:
dataset_artifact = ln.Artifact(zarr_filepath, description="Test dataset").save()

This is the where the zarr folder sits on a public S3 bucket:

In [ ]:
dataset_artifact.path.to_url()

Create a `VitessceConfig` object: 

In [ ]:
vc = VitessceConfig(
    schema_version="1.0.15",
    name="Test dataset",
    description="Test description",
)
dataset = vc.add_dataset(name="test1").add_object(
    AnnDataWrapper(
        adata_url=dataset_artifact.path.to_url(),
        obs_embedding_paths=["obsm/X_umap"],
    ),
)
obs_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(cm.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
vc.layout(((scatterplot | obs_sets) / heatmap) | (obs_sets_sizes / genes))
vc.to_dict()

In [ ]:
vitessce_config_artifact = ln.integrations.save_vitessce_config(vc, description="View testdata in Vitessce")

In [ ]:
# different equivalent ways of testing that the action is attached
assert dataset_artifact._actions.get() == vitessce_config_artifact
assert vitessce_config_artifact._action_targets.get() == dataset_artifact
assert vitessce_config_artifact._actions.first() is None
assert ln.Artifact.filter(_actions=vitessce_config_artifact).one() == dataset_artifact

In [ ]:
dataset_artifact.delete(permanent=True)
vitessce_config_artifact.delete(permanent=True)

In [ ]:
ln.setup.delete("test-vitessce", force=True)